## Setting up environment & importing needed libraries

For this I only have a CPU, so this is going to take some time 😭.

In [44]:
#Uncomment to install dependencies; I did this to avoid re-installing every time
!pip install transformers timm scikit-learn torch torchvision datasets pandas gdown zipfile36

  Using cached PySocks-1.7.1-py3-none-any.whl.metadata (13 kB)
Using cached PySocks-1.7.1-py3-none-any.whl (16 kB)

   -------- ------------------------------- 1/5 [soupsieve]
   ---------------- ----------------------- 2/5 [PySocks]
   ------------------------ --------------- 3/5 [beautifulsoup4]
   ------------------------ --------------- 3/5 [beautifulsoup4]
   ------------------------ --------------- 3/5 [beautifulsoup4]
   -------------------------------- ------- 4/5 [gdown]
   -------------------------------- ------- 4/5 [gdown]
   -------------------------------- ------- 4/5 [gdown]
   ---------------------------------------- 5/5 [gdown]



In [7]:
import json
import os
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from sklearn.metrics import classification_report, accuracy_score
from transformers import AutoModel, AutoTokenizer, get_scheduler
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from torch.optim import AdamW
from tqdm.notebook import tqdm, trange
from time import perf_counter
from PIL import Image
import pandas as pd

c:\Users\nirva\OneDrive\Projects\multimodal\multimodal-exploring\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Common configurations to be used throughout the notebook

In [8]:
# Check if GPU is available

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [9]:
import numpy as np
import random

# Function to set random seed for reproducibility

def set_seed(seed_val):

    random.seed(seed_val)

    np.random.seed(seed_val)

    torch.manual_seed(seed_val)

    torch.cuda.manual_seed_all(seed_val)

In [14]:
from PIL import Image
import requests
from io import BytesIO

def display_image_from_url(image_url):
    
    """

    Fetches an image from a URL and displays it.

    Args:
        image_url (str): The URL of the image.

    """
    try:

        response = requests.get(image_url)
        response.raise_for_status()  
        img = Image.open(BytesIO(response.content))
        img.show()

    except requests.exceptions.RequestException as e:

        print(f"Error fetching image from URL: {e}")

    except IOError as e:

        print(f"Error opening or displaying image: {e}")

In [17]:
seed_val = 67 # 6 7 vs. 41

set_seed(seed_val)

if seed_val == 67:

    # 67 - I feel so childish rn lol

    display_image_from_url("https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRD1tLi6s_qGP24DiHYjBiWGFkWcl-cMfeKcA&s")

print(f"Random seed set to: {seed_val}")

Random seed set to: 67


### Data loading & training parameters

This part downloads the data which is going to be used for the rest of the notebook & it defines the paths to read data from, as well as training parameters we are going to be using for all three models.

In [33]:
HOME_FOLDER = '' # Change this to your home folder if needed
WEBVISION_DATA_FOLDER = HOME_FOLDER + 'webvision_data/'
IMAGE_FOLDER = WEBVISION_DATA_FOLDER + 'images/'
RESULTS_FOLDER = HOME_FOLDER + 'results/'
TRAINED_MODELS_FOLDER = HOME_FOLDER + 'trained_models/'
os.makedirs(RESULTS_FOLDER, exist_ok=True)

In [45]:
import os, gdown

os.makedirs(WEBVISION_DATA_FOLDER, exist_ok=True)

url = "https://drive.google.com/uc?id=1r4aTTbLuYgGrgpZLogUH9sQ33DBsbOfM"
output = os.path.join(WEBVISION_DATA_FOLDER, "data.zip")

gdown.download(url, output, quiet=False)

with zipfile.ZipFile(output, 'r') as zip_ref:
    zip_ref.extractall(WEBVISION_DATA_FOLDER)


FileURLRetrievalError: Failed to retrieve file url:

	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses.
	Check FAQ in https://github.com/wkentaro/gdown?tab=readme-ov-file#faq.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=1r4aTTbLuYgGrgpZLogUH9sQ33DBsbOfM

but Gdown can't. Please check connections and permissions.